In [23]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from utils import calculate_gradient_and_update, cnn_train_step, cnn_val_step, save_checkpoint
from dataloader import CustomDataloader
from networks import CNNRegression, MultiModalNetwork
import tqdm

#### Automatic GPU optimization, if available

In [24]:
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", cuda_available)

Is CUDA available?  True


In [25]:
if (torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('GPU not available, using CPU')
    

Using GPU: NVIDIA GeForce GTX 1660 Ti


### Model 1 - Linear Regression ###

In [26]:
#load train test and validation data from tensor_collection
cm_x_train = torch.load('tensor_collection\\cm_x_train.pt')
cm_y_train = torch.load('tensor_collection\\cm_y_train.pt')
cm_x_test = torch.load('tensor_collection\\cm_x_test.pt')
cm_y_test = torch.load('tensor_collection\\cm_y_test.pt')
cm_x_val = torch.load('tensor_collection\\cm_x_val.pt')
cm_y_val = torch.load('tensor_collection\\cm_y_val.pt')



#### Hyperparameters setup

In [27]:
num_features = cm_x_train.shape[1]

train_loss_history = []
val_loss_history = []
num_epochs = 100
alpha = 0.1

# Initialize theta to random values between -2 and 2
theta = np.random.uniform(-2, 2, (num_features))


#### Model 1 - Training Loop 

In [28]:

# Training Loop
for t in range(num_epochs):
    # Training step
    train_loss, theta = calculate_gradient_and_update(cm_x_train, cm_y_train, theta, alpha)
    train_loss_history.append(train_loss)

    # Validation step
    val_loss, _ = calculate_gradient_and_update(cm_x_val, cm_y_val, theta, 0)  # alpha=0 to prevent updates
    val_loss_history.append(val_loss)


In [29]:
#save theta and loss history for analysis notebook
theta_torch = torch.tensor(theta, dtype=torch.float32)
train_loss_history_torch = torch.tensor(train_loss_history, dtype=torch.float32)
val_loss_history_torch = torch.tensor(val_loss_history, dtype=torch.float32)

torch.save(theta_torch, 'tensor_collection\\lr_theta.pt')
torch.save(train_loss_history_torch, 'tensor_collection\\lr_train_loss.pt')
torch.save(val_loss_history_torch, 'tensor_collection\\lr_val_loss.pt')


### Model 2 - Training CNN ###

#### Load in Train, Val, and Test sets for Model 2

In [30]:
directory = 'tensor_collection\\'

#load train and val csv files
cnn_x_train = pd.read_csv(directory + 'cnn_x_train.csv')
cnn_x_val = pd.read_csv(directory + 'cnn_x_val.csv')
cnn_y_train_tensor = torch.load(directory + 'cnn_y_train.pt')
cnn_y_val_tensor = torch.load(directory + 'cnn_y_val.pt')

print(type(cnn_x_train))
print(type(cnn_y_train_tensor))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


#### Instantiating Train and Val Dataloaders

In [31]:
train_dataloader = CustomDataloader(dataframe=cnn_x_train, age=cnn_y_train_tensor, batch_size=50, randomize=True)
val_dataloader = CustomDataloader(dataframe=cnn_x_val, age=cnn_y_val_tensor, batch_size=50, randomize=False)

#### Model 2 - Hyperparameters

In [32]:

train_losses = []
val_losses = []

model = CNNRegression().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 50
loss_fn = nn.MSELoss()

In [33]:


# Training Loop
for epoch in tqdm.tqdm(range(epochs)):
    # Training step
    train_loss = cnn_train_step(model, train_dataloader, loss_fn, optimizer, device)
    train_losses.append(train_loss)
    
    # Validation step
    val_loss = cnn_val_step(model, val_dataloader, loss_fn, device)
    val_losses.append(val_loss)

    save_checkpoint(model, optimizer, epoch, f'model_checkpoints/cnn_checkpoint_epoch_{epoch}.pth')

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\cnn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\cnn_val_losses.pt')




### Model 3 - Training Multi-Modal Network ###

#### Instantiating Train and Val Data loaders

In [ ]:
train_dataloader = CustomDataloader(dataframe=cnn_x_train, age=cnn_y_train_tensor, batch_size=50, randomize=True)
val_dataloader = CustomDataloader(dataframe=cnn_x_val, age=cnn_y_val_tensor, batch_size=50, randomize=False)


#### Model 3 - Hyperparameters

In [ ]:
num_numerical_features = cnn_x_train.shape[1] - 1



model = MultiModalNetwork(num_numerical_features).to(device)  # Adjust for the number of numerical features
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train_losses = []
val_losses = []
epochs = 10 #after 10 epochs the model train and validation loss began to spike
loss_fn = torch.nn.MSELoss()


In [ ]:




for epoch in tqdm.tqdm(range(epochs)):
    model.train()  # Set the model to training mode
    train_losses_epoch = []
    for _ in range(train_dataloader.num_batches_per_epoch):
        optimizer.zero_grad()
        train_batch = train_dataloader.fetch_batch()
        x_batch = train_batch['img_batch'].to(device)
        num_features = train_batch['feat_batch'].to(device)
        y_batch = train_batch['age_batch'].to(device)
        
        yhat = model(x_batch, num_features)
        yhat = torch.squeeze(yhat)
        train_loss = loss_fn(yhat, y_batch)
        train_loss.backward()
        optimizer.step()
        train_losses_epoch.append(train_loss.item())

    train_losses.append(np.mean(train_losses_epoch))  # Average loss for this epoch
    
    model.eval()  # Set the model to evaluation mode
    val_losses_epoch = []
    with torch.no_grad():
        for _ in range(val_dataloader.num_batches_per_epoch):
            val_batch = val_dataloader.fetch_batch()
            x_batch = val_batch['img_batch'].to(device)
            num_features = val_batch['feat_batch'].to(device)
            y_batch = val_batch['age_batch'].to(device)
            
            yhat = model(x_batch, num_features)
            yhat = torch.squeeze(yhat)
            val_loss = loss_fn(yhat, y_batch)
            val_losses_epoch.append(val_loss.item())

    val_losses.append(np.mean(val_losses_epoch))  # Average validation loss for this epoch
    
    checkpoint_filename = f'model_checkpoints/mmn_checkpoint_epoch_{epoch}.pth'
    save_checkpoint(model, optimizer, epoch, checkpoint_filename)


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:41<00:00, 10.14s/it]


In [ ]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\mmn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\mmn_val_losses.pt')
#store epochs for plotting
torch.save(epochs, 'tensor_collection\\mmn_epochs.pt')

